# Assignment 2
### Develop a model, to predict the column ‘GT Turbine decay state coefficient’

In [ ]:
import numpy as np
import pandas as pd
import math
import re
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from xgboost.sklearn import XGBRFRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

from sklearn.metrics import mean_absolute_error 
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV


In [ ]:
path = r"C:\\Users\\91969\\Desktop\\internship Project\\propulsion.csv"

data = pd.read_csv(path)
data

In [ ]:
Compressor_data = data.drop(['GT Turbine decay state coefficient.'] , axis = 1)
Turbine_data = data.drop(['GT Compressor decay state coefficient.'] , axis = 1)

In [ ]:
# split Compressor_data in train and Test

turb_train , turb_test = train_test_split(Turbine_data , test_size = 0.2 , random_state = 2)

In [ ]:
x_train = turb_train.drop('GT Turbine decay state coefficient.',axis = 1)
y_train = turb_train['GT Turbine decay state coefficient.']

In [ ]:
x_test = turb_test.drop('GT Turbine decay state coefficient.',axis = 1)

# LinearRegression

In [ ]:
turb_linear = LinearRegression()

In [ ]:
turb_linear.fit(x_train ,y_train )

In [ ]:
x_test = turb_test.drop('GT Turbine decay state coefficient.',axis = 1)

In [ ]:
predicted_Turbine = turb_linear.predict(x_test)

In [ ]:
mae = mean_absolute_error(turb_test['GT Turbine decay state coefficient.'],predicted_Turbine )
mae
#0.001677170038646267

In [ ]:
mse = mean_squared_error(turb_test['GT Turbine decay state coefficient.'],predicted_Turbine )
rmse = math.sqrt(mse)
rmse
#0.0022283966710053334

# Ridge( LinearRegression)

In [ ]:
# Making the list of number for lambda values

lambdaa =np.linspace(0.001,0.009,100)

In [ ]:
turb_train.reset_index(inplace = True , drop = True )

In [ ]:
x_train = turb_train.drop('GT Turbine decay state coefficient.' , axis = 1) 
y_train = turb_train["GT Turbine decay state coefficient."]

In [ ]:
mae_list = []                                                            # KFold Cross Validation

for a in lambdaa :
    
    ridge = Ridge(fit_intercept= True , alpha = a)
    kf = KFold( n_splits=5)
    val_error = 0
    
    for train , test in kf.split(x_train):
        ridge.fit(x_train.loc[train] , y_train[train])
        p = ridge.predict (x_train.loc[test])
        val_error += mean_absolute_error(y_train[test] , p)
    
    mae_5cv = val_error /5
    
    print( a, ":" , mae_5cv)
    mae_list.extend([mae_5cv])

In [ ]:
best_alpha = lambdaa[mae_list == min(mae_list)]

print("Alpha with minimum 5cv error is :" , best_alpha)
#Alpha with minimum 5cv error is : [0.001]

In [ ]:
ridge = Ridge(fit_intercept=True , alpha= best_alpha)

ridge.fit(x_train , y_train)

In [ ]:
# Now , To check Performance  on Test

x_test = turb_test.drop('GT Turbine decay state coefficient.' , axis = 1)

In [ ]:
predicted_test = ridge.predict(x_test)

In [ ]:
mae_ridge = mean_absolute_error(turb_test["GT Turbine decay state coefficient."] , predicted_test)
mae_ridge
#0.001676697434924658

In [ ]:
mse = mean_squared_error(turb_test['GT Turbine decay state coefficient.'],predicted_test )
rmse_ridge = math.sqrt(mse)
rmse_ridge
#0.002233530761662372

# Lasso ( LinearRegression)

In [ ]:
# Making the list of number for lambda values

lambdaa =np.linspace(1,100,100)

In [ ]:
turb_train.reset_index(inplace = True , drop = True )

In [ ]:
x_train = turb_train.drop('GT Turbine decay state coefficient.' , axis = 1) 
y_train = turb_train["GT Turbine decay state coefficient."]

In [ ]:
mae_list = []

for a in lambdaa :
    
    lasso = Lasso(fit_intercept= True , alpha = a)
    kf = KFold( n_splits=5)
    val_error = 0
    
    for train , test in kf.split(x_train):
        lasso.fit(x_train.loc[train] , y_train[train])
        p = lasso.predict (x_train.loc[test])
        val_error += mean_absolute_error(y_train[test] , p)
    
    mae_5cv = val_error /5
    
    print( a, ":" , mae_5cv)
    mae_list.extend([mae_5cv])

In [ ]:
best_alpha = lambdaa[mae_list == min(mae_list)]

print("Alpha with minimum 5cv error is :" , best_alpha)
# "Alpha with minimum 5cv error is :" , Multiple value

In [ ]:
lasso = Lasso(fit_intercept=True , alpha= best_alpha.all())

lasso.fit(x_train , y_train)

In [ ]:
# Now , To check Performance  on Test

x_test = turb_test.drop('GT Turbine decay state coefficient.' , axis = 1)

In [ ]:
predicted_test = lasso.predict(x_test)

In [ ]:
mae_lasso = mean_absolute_error(turb_test["GT Turbine decay state coefficient."] , predicted_test)
mae_lasso
#0.0065847900000000105

In [ ]:
mse = mean_squared_error(turb_test['GT Turbine decay state coefficient.'],predicted_test )
rmse_lasso = math.sqrt(mse)
rmse_lasso
#0.007568272822488971

# Decision Tree

In [ ]:
dtree = tree.DecisionTreeRegressor()

In [ ]:
dtree.fit(x_train , y_train)

In [ ]:
predicted_test = dtree.predict(x_test)

In [ ]:
mae = mean_absolute_error(turb_test["GT Turbine decay state coefficient."] , predicted_test)
mae
#0.0005694429581590133

In [ ]:
mse = mean_squared_error(turb_test['GT Turbine decay state coefficient.'],predicted_test )
rmse = math.sqrt(mse)
rmse
#0.0013011748998733941

# Hyper parameter for Decision tree

In [ ]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.5f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [ ]:
params={'criterion':['mae'],
        'max_depth':[None,5,10,15,20,30,50,70],
        'min_samples_leaf':[1,2,5,10,15,20], 
        'min_samples_split':[2,5,10,15,20],
        'max_features':[5,10,15,None],
        'max_leaf_nodes':[5,10,15,20,25]
       }

In [ ]:
decision_Regre = tree.DecisionTreeRegressor()

In [ ]:
Grid_search = GridSearchCV(decision_Regre, cv=5,
                           param_grid=params,
                           n_jobs=-1 , verbose=10)

In [ ]:
Grid_search.fit(x_train,y_train)

In [ ]:
Grid_search.best_estimator_
#DecisionTreeRegressor(criterion='mae', max_depth=50, max_features=15,
#                     max_leaf_nodes=25, min_impurity_decrease=0.0,
#                     min_impurity_split=None, min_samples_leaf=5,
#                     min_samples_split=15, min_weight_fraction_leaf=0.0,
#                     presort=False, random_state=None, splitter='best')

In [ ]:
report(Grid_search.cv_results_,3)

In [ ]:
dtree=Grid_search.best_estimator_

In [ ]:
dtree.fit(x_train,y_train)

In [ ]:
# Now , To check Performance  on Test

x_test = turb_test.drop('GT Turbine decay state coefficient.' , axis = 1)

In [ ]:
predicted_test = dtree.predict(x_test)

In [ ]:
mae_ = mean_absolute_error(turb_test["GT Turbine decay state coefficient."] , predicted_test)
mae_
## mae = 0.003475000000000006

In [ ]:
mse = mean_squared_error(turb_test['GT Turbine decay state coefficient.'],predicted_test )
rmse = math.sqrt(mse)
rmse
## rmse = 0.003475000000000006

# Random Forest

In [ ]:
ran_for_reg = RandomForestRegressor(criterion='mae' , verbose = 1 , n_jobs=-1)

In [ ]:
ran_for_reg.fit(x_train , y_train)

In [ ]:
predict = ran_for_reg.predict(x_test)

In [ ]:
mae =mean_absolute_error(turb_test["GT Turbine decay state coefficient."] , predict)
mae
## mae = 0.0005354000000000067

In [ ]:
mse = mean_squared_error(turb_test["GT Turbine decay state coefficient."], predict)
rmse = math.sqrt(mse)
print(rmse)
## rmse = 0.0010959881386219468

# Random Forest sequential Hyperparameter

In [ ]:
x_train.reset_index(drop=True,inplace=True)
y_train.reset_index(drop=True,inplace=True)

In [ ]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.5f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

# 1st

In [ ]:
rf_params = {  
                "n_estimators":[100,500,700,900,1000,1200,1500]
             }

In [ ]:
rf1=RandomForestRegressor(max_leaf_nodes=25)

In [ ]:
grid_search=GridSearchCV(rf1 ,cv=5 , param_grid=rf_params, verbose=10 , n_jobs=-1)

In [ ]:
grid_search.fit(x_train,y_train)

In [ ]:
report(grid_search.cv_results_,3)
## n_estimators = 700

# 2nd

In [ ]:
rf_params={
            "max_depth": [3,5,10,20,None],
            }

In [ ]:
rf2=RandomForestRegressor(max_leaf_nodes=25 , n_estimators= 700)

In [ ]:
grid_search=GridSearchCV(rf2,cv=5 , param_grid=rf_params , verbose=10 ,n_jobs=-1)

In [ ]:
grid_search.fit(x_train,y_train)

In [ ]:
report(grid_search.cv_results_,3)
## max_depth = 20

# 3rd

In [ ]:
rf_params={
            'min_samples_leaf':[1,5,10,15,20,25,30,35],
            'min_samples_split':[2,5,10,15,20]
            }

In [ ]:
rf3= RandomForestRegressor(max_leaf_nodes= 25 , n_estimators=700 ,max_depth = 20  )

In [ ]:
grid_search=GridSearchCV(rf3,param_grid=rf_params,cv=5 ,n_jobs=-1,verbose=10)

In [ ]:
grid_search.fit(x_train , y_train)

In [ ]:
report(grid_search.cv_results_,3)
## min_sample_leaf =5 , min_sample_split =15

# 4th

In [ ]:
rf_params={
            'bootstrap':[True,False]
            }

In [ ]:
rf4= RandomForestRegressor(max_leaf_nodes=25 , n_estimators=700  ,max_depth = 20 ,min_samples_leaf = 5 ,min_samples_split =15 )

In [ ]:
grid_search=GridSearchCV(rf4,param_grid=rf_params,cv=5 ,n_jobs=-1,verbose=10)

In [ ]:
grid_search.fit(x_train , y_train)

In [ ]:
report(grid_search.cv_results_,3)
## bootstrap = True

# 5th

In [ ]:
rf_params={
           'max_leaf_nodes':[20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35]
            }

In [ ]:
rf5= RandomForestRegressor(n_estimators=700  ,max_depth = 20,min_samples_leaf = 5,min_samples_split =15 ,
                           bootstrap =True )

In [ ]:
grid_search=GridSearchCV(rf5,param_grid=rf_params,cv=5 ,n_jobs=-1,verbose=10)

In [ ]:
grid_search.fit(x_train , y_train)

In [ ]:
report(grid_search.cv_results_,3)
## max_leaf_nodes = 35

# 6th (final)

In [ ]:
grid_search.best_estimator_

In [ ]:
report(grid_search.cv_results_,3)

In [ ]:
rf6 = RandomForestRegressor( bootstrap=True, criterion='mse', max_depth=20,
                      max_features='auto', max_leaf_nodes=35,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=5, min_samples_split=15,
                      min_weight_fraction_leaf=0.0, n_estimators=700,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False )

In [ ]:
rf6.fit(x_train , y_train)

In [ ]:
predict = rf6.predict(x_test)

In [ ]:
mae = mean_absolute_error(turb_test["GT Turbine decay state coefficient."] , predict)
mae
## mae = 0.0025016475396092675

In [ ]:
mse = mean_squared_error(turb_test['GT Turbine decay state coefficient.'], predict)
rmse = math.sqrt(mse)
rmse
## rmse = 0.0031944400513935446

# ------------------------- Boosting -----------------------------

# AdaBoost Regresser

In [ ]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.5f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [ ]:
x_train.reset_index(drop=True,inplace=True)
y_train.reset_index(drop=True,inplace=True)

In [ ]:
base_rf=RandomForestRegressor( max_leaf_nodes= 35)

In [ ]:
adb=AdaBoostRegressor(base_estimator=base_rf)

In [ ]:
adb_params={'n_estimators':[50,100,200,500,700],
           'learning_rate': [0.01,.1,1]
           }

In [ ]:
complete_search= GridSearchCV(adb, param_grid=adb_params, cv=5 , n_jobs=-1 , verbose=10)

In [ ]:
complete_search.fit(x_train , y_train)

In [ ]:
report(complete_search.cv_results_)
## learning_rate = 1 , n_estimators = 700

In [ ]:
complete_search.best_estimator_
#AdaBoostRegressor(base_estimator=RandomForestRegressor(bootstrap=True,criterion='mse', max_depth=None,max_features='auto',
#                                                      max_leaf_nodes=35,min_impurity_decrease=0.0,min_impurity_split=None,
#                                                       min_samples_leaf=1,min_samples_split=2,min_weight_fraction_leaf=0.0,
#                                                       n_estimators='warn',n_jobs=None,oob_score=False,random_state=None,
#                                                   verbose=0,warm_start=False),learning_rate=1, loss='linear', n_estimators=700,
#                                                      random_state=None)

In [ ]:
adb_best=AdaBoostRegressor(base_estimator=base_rf,learning_rate= 1 ,n_estimators= 700 )

In [ ]:
adb_best.fit(x_train,y_train)

In [ ]:
predict = adb_best.predict(x_test)

In [ ]:
mae = mean_absolute_error(turb_test["GT Turbine decay state coefficient."] , predict)
mae
## mae = 0.0016421447873783094

In [ ]:
mse = mean_squared_error(turb_test['GT Turbine decay state coefficient.'], predict)
rmse = math.sqrt(mse)
rmse
## rmse = 0.0020035366616068325

# XGBoost With sequential Parameter

In [ ]:
import re

In [ ]:
regex = re.compile(r"\[|\]|<", re.IGNORECASE)

In [ ]:
data.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in data.columns.values]

In [ ]:
x_train.reset_index(drop=True,inplace=True)
y_train.reset_index(drop=True,inplace=True)

In [ ]:
xgb_params = {  
             "n_estimators":[100,500,700,900,1000,1200,1500]
             }

In [ ]:
xgb1=XGBRFRegressor(learning_rate = 0.1 ,subsample = 0.8 ,colsample_bylevel = 0.8 , colsample_bytree = 0.8)

In [ ]:
grid_search=GridSearchCV(xgb1 , cv=5 , param_grid = xgb_params , verbose=10 , n_jobs=-1 )

In [ ]:
grid_search.fit(x_train,y_train)

In [ ]:
report(grid_search.cv_results_,3)
## 'n_estimators': 500

# 2nd

In [ ]:
xgb_params={
            "gamma":[0,2,5,8,10],
            "max_depth": [2,3,4,5,6,7,8],
            "min_child_weight":[0.5,1,2,5,10]
            }

In [ ]:
xgb2=XGBRFRegressor(learning_rate=0.1,n_estimators=500,subsample=0.8,colsample_bylevel=0.8,colsample_bytree=0.8)

In [ ]:
random_search=GridSearchCV(xgb2,param_grid=xgb_params,cv=5,                             
                                 n_jobs=-1,verbose=10)                                               

In [ ]:
random_search.fit(x_train,y_train)

In [ ]:
report(random_search.cv_results_,3)
## 'gamma': 0, 'max_depth': 8, 'min_child_weight': 0.5

# 3rd

In [ ]:
xgb_params={
            'max_delta_step':[0,1,3,6,10],
            'scale_pos_weight':[1,2,3,4]
            }

In [ ]:
xgb3=XGBRFRegressor(learning_rate=0.1,n_estimators=500 ,min_child_weight=0.5,gamma=0,max_depth=8,
                  subsample=0.8,colsample_bylevel=0.8,colsample_bytree=0.8)

In [ ]:
grid_search=GridSearchCV(xgb3,param_grid=xgb_params,cv=5,n_jobs=-1,verbose=10)

In [ ]:
grid_search.fit(x_train,y_train)

In [ ]:
report(grid_search.cv_results_,3)
## 'max_delta_step': 0, 'scale_pos_weight': 4

# 4th

In [ ]:
xgb_params={
            'subsample':[i/10 for i in range(5,11)],
            'colsample_bytree':[i/10 for i in range(5,11)],
            'colsample_bylevel':[i/10 for i in range(5,11)]
            }

In [ ]:
xgb4=XGBRFRegressor(learning_rate=0.1,n_estimators=500 ,min_child_weight=0.5,gamma=0,max_depth=8,
                        scale_pos_weight=4,max_delta_step=0
                  )

In [ ]:
random_search=GridSearchCV(xgb4,param_grid=xgb_params,cv=5,                             
                                n_jobs=-1,verbose=10)

In [ ]:
random_search.fit(x_train,y_train)

In [ ]:
report(random_search.cv_results_,3)
## 'colsample_bylevel': 0.5, 'colsample_bytree': 0.5, 'subsample': 1.0

# 5th

In [ ]:
xgb5=XGBRFRegressor(learning_rate=0.1,n_estimators=500,min_child_weight=0.5,gamma=0,max_depth=8,
                        scale_pos_weight=4,max_delta_step=0,
                   colsample_bylevel=0.5 , colsample_bytree= 0.5, subsample= 1.0
                  )

In [ ]:
xgb_params={
            'reg_lambda':[i/10 for i in range(0,50)],
            'reg_alpha':[i/10 for i in range(0,50)]
            }

In [ ]:
random_search=GridSearchCV(xgb5,param_grid=xgb_params,cv=5,                                  
                                n_jobs=-1,verbose=10)

In [ ]:
random_search.fit(x_train,y_train)

In [ ]:
report(random_search.cv_results_,3)
## 'reg_alpha': 0.0, 'reg_lambda': 0.1

# 6th

In [ ]:
random_search.best_estimator_
#XGBRFRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.5,
#              colsample_bynode=0.8, colsample_bytree=0.5, gamma=0, gpu_id=-1,
#              importance_type='gain', interaction_constraints='',
#              learning_rate=0.1, max_delta_step=0, max_depth=8,
#              min_child_weight=0.5, missing=nan, monotone_constraints='()',
#              n_estimators=500, n_jobs=0, num_parallel_tree=500,
#              objective='reg:squarederror', random_state=0, reg_alpha=0.0,
#              reg_lambda=0.1, scale_pos_weight=4, subsample=1.0,
#              tree_method='exact', validate_parameters=1, verbosity=None)

In [ ]:
xgb6=XGBRFRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.5,
               colsample_bynode=0.8, colsample_bytree=0.5, gamma=0, gpu_id=-1,
               importance_type='gain', interaction_constraints='',
               learning_rate=0.1, max_delta_step=0, max_depth=8,
               min_child_weight=0.5,  monotone_constraints='()',
               n_estimators=500, n_jobs=-1, num_parallel_tree=500,
               objective='reg:squarederror', random_state=0, reg_alpha=0.0,
               reg_lambda=0.1, scale_pos_weight=4, subsample=1.0,
               tree_method='exact', validate_parameters=1, verbosity=None )

In [ ]:
xgb6.fit(x_train , y_train)

In [ ]:
predict = xgb6.predict(x_test)

In [ ]:
mae = mean_absolute_error(turb_test["GT Turbine decay state coefficient."] , predict)
mae
## mae = 0.4385821460032463

In [ ]:
mse = mean_squared_error(turb_test['GT Turbine decay state coefficient.'], predict)
rmse = math.sqrt(mse)
rmse
## rmse = 0.43864731421871933

# GBM ( sequential hyperparameter )

In [ ]:
x_train.reset_index(drop=True,inplace=True)
y_train.reset_index(drop=True,inplace=True)

In [ ]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.5f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

# 1st

In [ ]:
gbm_params = {  
                "n_estimators":[10, 50,100,500,700,900,1000,1200,1500]
             }

In [ ]:
gbm1=GradientBoostingRegressor(max_leaf_nodes=25)

In [ ]:
grid_search=GridSearchCV(gbm1,cv=5,param_grid=gbm_params,verbose=10,n_jobs=-1)

In [ ]:
grid_search.fit(x_train , y_train)

In [ ]:
report(grid_search.cv_results_,3)
## 'n_estimators': 1500

# 2nd

In [ ]:
gbm_params={
            "max_depth": [1,2,3,5,10,20,None],
            }

In [ ]:
gbm2=GradientBoostingRegressor(max_leaf_nodes=25 , n_estimators= 1500 )

In [ ]:
grid_search=GridSearchCV(gbm2,cv=5,param_grid=gbm_params,verbose=10 ,n_jobs=-1)

In [ ]:
grid_search.fit(x_train,y_train)

In [ ]:
report(grid_search.cv_results_,3)
## 'max_depth': 3

# 3rd

In [ ]:
gbm_params={
            'min_samples_leaf':[1,5,10,15,20,25],
            'min_samples_split':[2,5,10,15,20]
            }

In [ ]:
gbm3= GradientBoostingRegressor(max_leaf_nodes=25 , n_estimators=1500 ,max_depth = 3)

In [ ]:
grid_search=GridSearchCV(gbm3,param_grid=gbm_params,cv=5,n_jobs=-1,verbose=10)

In [ ]:
grid_search.fit(x_train , y_train)

In [ ]:
report(grid_search.cv_results_,3)
## 'min_samples_leaf': 20, 'min_samples_split': 20

# 4th

In [ ]:
gbm_params={
            'learning_rate':[0.01,0.05,0.1,0.4,0.8,1],
            'subsample': [0.5,0.8,1]
            }

In [ ]:
gbm4= GradientBoostingRegressor(max_leaf_nodes=25 , n_estimators=1500  ,max_depth =3,
                                min_samples_leaf =20 ,min_samples_split =20 )

In [ ]:
grid_search=GridSearchCV(gbm4,param_grid=gbm_params,cv=5,n_jobs=-1,verbose=10)

In [ ]:
grid_search.fit(x_train , y_train)

In [ ]:
report(grid_search.cv_results_,3)
## 'learning_rate': 0.1, 'subsample': 0.8

# 5th

In [ ]:
gbm_params={
           'max_leaf_nodes':[20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35],
           'max_features':[5,10,None]
            }

In [ ]:
gbm5= GradientBoostingRegressor(n_estimators= 1500 ,max_depth =3 ,min_samples_leaf =20 ,min_samples_split =20 ,
                           learning_rate= 0.1,subsample= 0.8 )

In [ ]:
grid_search=GridSearchCV(gbm5 , param_grid=gbm_params ,cv=5 , n_jobs=-1 , verbose=10)

In [ ]:
grid_search.fit(x_train , y_train)

In [ ]:
report(grid_search.cv_results_,5)
## 'max_features': None, 'max_leaf_nodes': 30

# 6th

In [ ]:
grid_search.best_estimator_
#alpha=0.9, criterion='friedman_mse', init=None,
#                         learning_rate=0.1, loss='ls', max_depth=3,
#                         max_features=None, max_leaf_nodes=30,
#                         min_impurity_decrease=0.0, min_impurity_split=None,
#                         min_samples_leaf=20, min_samples_split=20,
#                         min_weight_fraction_leaf=0.0, n_estimators=1500,
#                         n_iter_no_change=None, presort='auto',
#                         random_state=None, subsample=0.8, tol=0.0001,
#                         validation_fraction=0.1, verbose=0, warm_start=False

In [ ]:
gbm6= GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=30,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=20, min_samples_split=20,
                          min_weight_fraction_leaf=0.0, n_estimators=1500,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=0.8, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
gbm6.fit(x_train , y_train)

In [ ]:
# Now , To check Performance  on Test

x_test = turb_test.drop('GT Compressor decay state coefficient.' , axis = 1)

In [ ]:
predict = gbm6.predict(x_test)

In [ ]:
mae = mean_absolute_error(turb_test["GT Turbine decay state coefficient."] , predict)
mae
## mae = 0.0004676401751022908

In [ ]:
mse = mean_squared_error(turb_test['GT Turbine decay state coefficient.'], predict)
rmse = math.sqrt(mse)
rmse
## rmse = 0.0007173625714369948

# KNN

In [ ]:
knn = KNeighborsRegressor( n_neighbors=5)

In [ ]:
knn.fit(x_train , y_train) 

In [ ]:
predications = knn.predict(x_test)

In [ ]:
mae = mean_absolute_error(turb_test["GT Turbine decay state coefficient."] , predications)
mae
## mae = 0.006413400000000007

In [ ]:
mse = mean_squared_error(turb_test["GT Turbine decay state coefficient."],predications)
rmse = math.sqrt(mse)
rmse
## rmse = 0.0076280062926036025

# KNN ( Hyper parameter )

In [ ]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.5f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [ ]:
x_train.reset_index(drop=True,inplace=True)
y_train.reset_index(drop=True,inplace=True)

In [ ]:
param = { 'n_neighbors' : list(range(5,10,15)) ,
        'p' : [1,2,3],
        'weights': ['uniform' , 'distance'],
        'algorithm': ['auto' , 'ball_tree' , 'kd_tree' , 'brute']}

In [ ]:
gride = GridSearchCV(KNeighborsRegressor() , param_grid = param  , cv = 5 , n_jobs=-1 , verbose=10)

In [ ]:
gride.fit(x_train, y_train)

In [ ]:
gride.best_estimator_

In [ ]:
report(gride.cv_results_,3)

In [ ]:
knn=gride.best_estimator_

In [ ]:
knn.fit(x_train , y_train) 

In [ ]:
#x_test = comp_test.drop('GT Compressor decay state coefficient.' , axis = 1)

In [ ]:
predications = knn.predict(x_test)

In [ ]:
mae = mean_absolute_error(turb_test["GT Turbine decay state coefficient."] , predications)
mae
## mae = 0.0055643709629743686

In [ ]:
mse = mean_squared_error(turb_test["GT Turbine decay state coefficient."],predications)
rmse = math.sqrt(mse)
rmse
## rmse = 

# SVM( SVR)

In [ ]:
svr = SVR()

In [ ]:
svr.fit(x_train , y_train)

In [ ]:
predications = svr.predict(x_test)

In [ ]:
mae = mean_absolute_error(turb_test["GT Turbine decay state coefficient."] , predications)
mae
## mae = 

In [ ]:
mse = mean_squared_error(turb_test["GT Turbine decay state coefficient."],predications)
rmse = math.sqrt(mse)
rmse
## rmse = 0.0075684212356342965

# SVM ( SVR - Hyper parameter)

In [ ]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.5f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [ ]:
x_train.reset_index(drop=True,inplace=True)
y_train.reset_index(drop=True,inplace=True)

In [ ]:
param = {'C' : [0.01,0.1,1.0],
        'kernel' : ['linear' , 'poly' , 'rbf' , 'sigmoid'],
        'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
        'coef0': [0, 1]}

In [ ]:
gride = GridSearchCV(SVR() , param_grid = param  , cv = 5 , n_jobs=-1 , verbose=10)

In [ ]:
gride.fit(x_train, y_train)

In [ ]:
gride.best_estimator_

In [ ]:
report(gride.cv_results_,3)

In [ ]:
svr_=gride.best_estimator_

In [ ]:
svr_.fit(x_train , y_train) 

In [ ]:
predications = svr_.predict(x_test)

In [ ]:
mae = mean_absolute_error(turb_test["GT Turbine decay state coefficient."],predications)
mae
## mse = 0.006585500000000011

In [ ]:
mse = mean_squared_error(turb_test["GT Turbine decay state coefficient."],predications)
rmse = math.sqrt(mse)
rmse
## rmse = 0.0075684212356342965



# All the Results together for Comparison.

In [1]:
nesteslist = [['Linear Regression' , 0.0016771, 0.0022283] ,
              ['Ridge(Linear Regression)' , 0.0016766 , 0.0022335 ] ,
              ['Lasso(Linear Regression)' , 0.0065847 , 0.0075682] ,
              ['DecisionTree' , 0.0005694  , 0.0013011 ],
              ['DecisionTree(hyperparameter)' , 0.0034750 , 0.0046038 ],
              ['RandomForest' , 0.00053540 , 0.0010959 ] ,
              ['RandomForest(hyperparameter)' , 0.00250164 , 0.0031944],
              ['AdaBoost Regresser' , 0.00164214 , 0.0020035 ],
              ['GradientBoosting' , 0.0004676 , 0.0007173 ] ,
              ['XGBRegressor' , 0.4385821 ,  0.43864731] ,
              ['KNN' , 0.0064134 , 0.0076280 ],
              ['KNN(hyperparameter)' , 0.005564370 , 0.0067534 ],
              ['SVM' , 0.0065855, 0.00756842 ],
              ['SVM(hyperparameter)' , 0.00658550 , 0.00756842 ]]

In [2]:
print("|          Model Name           |           MAE           |         RMSE            |")
for item in nesteslist :
    print("|",item[0]," "*(28-len(item[0])),"|" ,
         item[1]," "*(22-len(str(item[1]))),"|",
         item[2]," "*(22-len(str(item[2]))),"|")

|          Model Name           |           MAE           |         RMSE            |
| Linear Regression             | 0.0016771               | 0.0022283               |
| Ridge(Linear Regression)      | 0.0016766               | 0.0022335               |
| Lasso(Linear Regression)      | 0.0065847               | 0.0075682               |
| DecisionTree                  | 0.0005694               | 0.0013011               |
| DecisionTree(hyperparameter)  | 0.003475                | 0.0046038               |
| RandomForest                  | 0.0005354               | 0.0010959               |
| RandomForest(hyperparameter)  | 0.00250164              | 0.0031944               |
| AdaBoost Regresser            | 0.00164214              | 0.0020035               |
| GradientBoosting              | 0.0004676               | 0.0007173               |
| XGBRegressor                  | 0.4385821               | 0.43864731              |
| KNN                           | 0.0064134           